**TODO**

- call API to get actualy PPET values to determine drought area
 - get centroids for each square
 - pass centroids to API
- get stats for droughted area (convex hull):
 - spatial join convex hull with original grid with PPET values
 - get values for all 9km x 9km squares in hull
 - do proportional split on squares that don't lie 100% in convex hull
 - get area
- get days over time period specified
- get centroid for hull

In [1]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

# for aWhere API
import requests as rq

from header import AWhereAPI
from secret_codes import *

In [ ]:
# Create df with 1 county
path_to_county = os.path.join(home, "Desktop", "drought-tracker", "data", "TIGER", "cb_2018_us_county_500k.shp")

counties = gpd.read_file(path_to_county)

county = counties.loc[counties.NAME == 'Siskiyou']

county = county.to_crs('epsg:2019')

In [ ]:
# Create df with 1 test county (Marin)
path_to_marin = os.path.join(home, "Desktop", "drought-tracker", "data", "TIGER", "marin.csv")

marin = gpd.read_file(path_to_marin)

marin

In [ ]:
def create_awhere_grid(crs, aoi):
    """
    Parameters
    ----------
    crs: numeric string
        String of EPSG code for best CRS
        CRS must have units of meters
        
        
    aoi: Geopandas GeoDataFrame
        One-element GeoDataFrame containing area over which
        to draw grid
        
    Returns
    -------
    GeoDataFrame containing 9km x 9km cells
    """   
    
    # Reproject aoi to CRS using meters for aWhere grid cells
    aoi = aoi.to_crs(f'epsg:{crs}')
    
    # Get x and y min and max from total boundaries
    xmin, ymin, xmax, ymax = aoi.total_bounds
    
    # Set side of grid cell to 9 km
    side = 9000
    
    # Create x values for x points for rows
    x_range = np.arange(int(np.floor(xmin)), int(np.ceil(xmax)), side)
    
    # Create y values
    y_range = np.arange(int(np.floor(ymin)), int(np.floor(ymax)), side)
    
    # Create empty list to hold grid cells
    polygons = []
    
    for x in x_range:
        for y in y_range:
            polygons.append(Polygon([(x,y), (x+side, y), (x+side, y+side), (x,y+side)]))
            
    grid = gpd.GeoDataFrame({'geometry': polygons})
    
    grid.crs = f'EPSG:{crs}'
    
    return grid

grid = create_awhere_grid('2019', county)

# Create plot to check data
fig, ax = plt.subplots()

county.plot(alpha=0.2, ax=ax)

grid.plot(color="none", edgecolor="black", linewidth = 0.1, ax=ax)

plt.show()

In [ ]:
# Add accumulated_ppet column to grid
grid = grid.assign(accumulated_ppet = 0)

In [ ]:
# Generate random values for each cell and add to grid gdf
grid['test_ppet'] = [np.random.randint(0,30) for i in range(0, grid.shape[0])]

In [ ]:
# Allow user to change threshold as daily value
daily_accumulated_ppet = 0.5

# Calculate monthly value
monthly_accumulated_ppet = daily_accumulated_ppet * 30

# Calcualte if PPET meets threshold and add Boolean flag
grid['meet_thresh'] = grid['test_ppet'].apply(lambda x: 1 if x >= monthly_accumulated_ppet else 0)

In [ ]:
# Drop all rows that do not meet threshold
droughted = grid[grid.meet_thresh == 1]

# Dissolve polygons
drought_dissolve = droughted.dissolve(by='meet_thresh')

In [ ]:
# Explode polygons so that there are unique contiguous areas
exploded = drought_dissolve.explode()

In [ ]:
# Add area column
exploded["sqkm"] = exploded['geometry'].area/ 10**6

# Drop multi-index outer level generated from dissolve
exploded = exploded.droplevel(0)

In [ ]:
# Get largest polygon
largest = exploded[exploded.sqkm == exploded.sqkm.max()]

In [ ]:
# Generate convex hull
hull_largest = largest.convex_hull

# Convert to gdf
hull = gpd.GeoDataFrame(hull_largest, crs='epsg:2019')

# Rename column '0' to 'geometry'
hull.rename(columns={0:"geometry"}, inplace=True)

In [ ]:
fig, ax = plt.subplots()


exploded.plot(ax=ax, column='sqkm')
#largest.plot(ax=ax)
#largest_envelope.plot(ax=ax, alpha=0.2)
#simple_largest.plot(ax=ax, alpha=0.4, color='orange')
hull.plot(ax=ax, alpha=0.4)

plt.show()